In [ ]:
### 입력으로 상호명을 받으면 업종분류를 반환하는 프로그램
# MeCab으로 미리 전처리한 후 불러와서 학습에 사용

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
import time
import os

from tqdm import tqdm
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from transformers import BertTokenizerFast, BertConfig, BertModel # Bert 모델과 토크나이저
from torch.utils.data import Dataset, DataLoader
from collections import Counter

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

c:\code\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
torch.cuda.init()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.reset_peak_memory_stats(device=None)
print("현재 디바이스:", device)

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

현재 디바이스: cuda


In [10]:
# 전처리된 CSV 파일 로드
def load_processed_data(file_path):
    """
    처리된 CSV 파일을 로드하는 함수
    """
    df = pd.read_csv(file_path)
    print(f"로드된 데이터 크기: {df.shape}")
    print(f"컬럼 목록: {df.columns.tolist()}")
    
    # 필요한 열만 추출 (업체명과 클래스)
    result_df = df[['상호명_Regulated', '클래스']].copy()
    
    # 컬럼명 통일
    result_df.columns = ['store', 'class']
    
    print(f"전처리 후 데이터 크기: {result_df.shape}")
    return result_df

In [11]:
data_df = load_processed_data('./processed_data/region_all_processed_data.csv')

로드된 데이터 크기: (1818487, 5)
컬럼 목록: ['상가업소번호', '지역', '상호명_Regulated', '업종소분류_Regulated', '클래스']
전처리 후 데이터 크기: (1818487, 2)


In [12]:
def split_dataset(df, test_size=0.2, random_state=42):
    """
    데이터를 훈련용과 테스트용으로 분할하는 함수
    """
    # 클래스 분포 확인
    num_classes = df['class'].nunique()
    print(f"고유 클래스 수: {num_classes}")
    
    # 데이터셋 분할 (stratify로 클래스 분포 유지)
    train_df, test_df = train_test_split(
        df, 
        test_size=test_size, 
        random_state=random_state,
        stratify=df['class']
    )
    
    print(f"훈련 데이터 크기: {train_df.shape}")
    print(f"테스트 데이터 크기: {test_df.shape}")

    return train_df, test_df

In [13]:
train_df, test_df = split_dataset(data_df)

고유 클래스 수: 247
훈련 데이터 크기: (1454789, 2)
테스트 데이터 크기: (363698, 2)


In [14]:
train_df
# test_df

,store,class
1607566,드림아트,80
1385651,온가족국수,35
313687,알로하훌라스튜디오,76
1811134,전자담배라미야김제점,117
1391266,몽키리코에프엔비완주봉동지점,214
...,...,...
354405,크린토피아홈플러스서,123
798530,신한철물,207
1446488,갈목철공소,11
971736,이사랑서울치과의원,211


In [15]:
train_df.groupby(by=['class']).count()

,store
class,
0,4147
1,6031
2,582
3,2262
4,1308
...,...
242,1275
243,734
244,14120


In [16]:
tokenizer = BertTokenizerFast.from_pretrained('kykim/bert-kor-base')
token1 = tokenizer.tokenize("봉암쇼핑")
token2 = tokenizer.tokenize("밀사랑손칼국수")
token3 = tokenizer.tokenize("태민건축적산사무소")
token4 = tokenizer.tokenize("스타벅스R리저브강남대로점")
token5 = tokenizer.tokenize("로그인커피")
token6 = tokenizer.tokenize("스타벅스R리저브강남속편한치과건너편점")
print(token1, token2, token3, token4, token5, token6)

mecab = Mecab(dicpath='C:/mecab/mecab-ko-dic')
token1 = mecab.morphs("봉암쇼핑")
token2 = mecab.morphs("밀사랑손칼국수")
token3 = mecab.morphs("태민건축적산사무소")
token4 = mecab.morphs("스타벅스R리저브강남대로점")
token5 = mecab.morphs("로그인커피")
token6 = mecab.morphs("스타벅스R리저브강남속편한치과건너편점")
print(token1, token2, token3, token4, token5, token6)

['봉', '##암', '##쇼핑'] ['밀', '##사랑', '##손', '##칼국수'] ['태', '##민', '##건축', '##적', '##산', '##사무소'] ['스타벅스', '##r', '##리', '##저', '##브', '##강', '##남', '##대로', '##점'] ['로그인', '##커피'] ['스타벅스', '##r', '##리', '##저', '##브', '##강', '##남', '##속', '##편한', '##치', '##과', '##건', '##너', '##편', '##점']
['봉암', '쇼핑'] ['밀사', '랑', '손칼국수'] ['태민', '건축', '적', '산', '사무소'] ['스타', '벅스', 'R', '리저브', '강남대로', '점'] ['로그인', '커피'] ['스타', '벅스', 'R', '리저브', '강남', '속편', '한', '치과', '건너편', '점']


In [17]:
# 토크나이저 세팅
mecab = Mecab(dicpath='C:/mecab/mecab-ko-dic')
bert_tokenizer = BertTokenizerFast.from_pretrained('kykim/bert-kor-base')
vocab = bert_tokenizer.get_vocab()  # huggingface vocab 사용

In [18]:
# mecab을 활용하여 bert tokenizer vocab에 맞게 변환
def mecab_tokenize_and_encode(sentence, vocab, max_length=60):
    tokens = mecab.morphs(str(sentence))
    # vocab에 없는 토큰은 [UNK]로 처리
    input_ids = [vocab.get('[CLS]', 2)]  # [CLS] 토큰
    for token in tokens:
        input_ids.append(vocab.get(token, vocab.get('[UNK]', 3)))
    input_ids.append(vocab.get('[SEP]', 3))  # [SEP] 토큰
    # 패딩
    if len(input_ids) < max_length:
        input_ids += [vocab.get('[PAD]', 0)] * (max_length - len(input_ids))
    else:
        input_ids = input_ids[:max_length]
    attention_mask = [1 if idx != vocab.get('[PAD]', 0) else 0 for idx in input_ids]
    token_type_ids = [0] * max_length
    return {
        'input_ids': torch.tensor(input_ids, dtype=torch.long),
        'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
        'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
    }

In [12]:
temp_word = mecab_tokenize_and_encode("스타벅스R리저브강남속편한치과건너편점", vocab) # 기존 tokenizer와 동일한 형식인지 확인
temp_word

{'input_ids': tensor([    2, 14314,     1,     1,     1, 16938,     1,  7653, 28026,     1,
          6030,     3,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [19]:
class TokenDataset_Mecab(Dataset):
    def __init__(self, pickle_path):
        self.data = pickle.load(open(pickle_path, 'rb'))
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        tokens, label = self.data[idx]
        return tokens, label

In [14]:
class TokenDataset(Dataset):
    def __init__(self, dataframe, tokenizer_pretrained):
        # sentence, label 컬럼으로 구성된 데이터프레임 전달
        self.data = dataframe        
        # Huggingface 토크나이저 생성
        self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)
  
    def __len__(self):
        return len(self.data)
  
    def __getitem__(self, idx):
        sentence = self.data.iloc[idx]['store']
        label = self.data.iloc[idx]['class']

        # 토큰화 처리
        tokens = self.tokenizer(
            str(sentence),                # 1개 문장 
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            max_length=60,          # 최대 길이 60
            add_special_tokens=True  # 스페셜 토큰 적용
        )

        input_ids = tokens['input_ids'].squeeze(0)           # 2D -> 1D
        attention_mask = tokens['attention_mask'].squeeze(0) # 2D -> 1D
        token_type_ids = torch.zeros_like(attention_mask)

        # input_ids, attention_mask, token_type_ids 이렇게 3가지 요소를 반환하도록 합니다.
        # input_ids: 토큰
        # attention_mask: 실제 단어가 존재하면 1, 패딩이면 0 (패딩은 0이 아닐 수 있습니다)
        # token_type_ids: 문장을 구분하는 id. 단일 문장인 경우에는 전부 0
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask, 
            'token_type_ids': token_type_ids,
        }, torch.tensor(label)

In [23]:
def preprocess_and_save(dataset_df, vocab, save_path):
    processed = []
    for idx, row in tqdm(dataset_df.iterrows(), total=len(dataset_df)):
        tokens = mecab_tokenize_and_encode(row['store'], vocab)
        label = torch.tensor(int(row['class']))
        processed.append((tokens, label))
    pickle.dump(processed, open(save_path, 'wb'))
    print(f"Saved {len(processed)} samples to {save_path}")

In [24]:
# 학습/테스트 데이터셋 MeCab으로 미리 전처리 및 저장
preprocess_and_save(train_df, vocab, './processed_data/train_mecab_tokenized.pkl')
preprocess_and_save(test_df, vocab, './processed_data/test_mecab_tokenized.pkl')

100%|██████████| 1454789/1454789 [01:52<00:00, 12982.48it/s]


Saved 1454789 samples to ./processed_data/train_mecab_tokenized.pkl


100%|██████████| 363698/363698 [00:22<00:00, 16308.89it/s]


Saved 363698 samples to ./processed_data/test_mecab_tokenized.pkl


In [25]:
train_data = TokenDataset_Mecab('./processed_data/train_mecab_tokenized.pkl')
test_data = TokenDataset_Mecab('./processed_data/test_mecab_tokenized.pkl')

# DataLoader로 이전에 생성한 Dataset를 지정하여, batch 구성, shuffle, num_workers 등을 설정합니다.
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=0)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True, num_workers=0)

In [31]:
train_data

In [32]:
# 1개의 batch 꺼내기
inputs, labels = next(iter(train_loader))

# 데이터셋을 device 설정
inputs = {k: v.to(device) for k, v in inputs.items()}
labels.to(device)

tensor([237, 158, 229, 107,  68, 158,  41, 242,  34,  86, 203, 146, 180,  91,
         94, 146, 158, 191, 214,  61,  87,  61,  91,  91,  21,  49, 184, 173,
        125,  62,  22, 182,  20,  55, 151, 214,  47, 125,  17,  33,  33,  12,
         32, 103, 123,  52,  24, 207, 131, 212, 122,  39, 229,  35,  60, 225,
        214, 178, 214,  87, 107, 103,  67, 229], device='cuda:0')

In [33]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'token_type_ids'])

In [34]:
inputs['input_ids'].shape, inputs['attention_mask'].shape, inputs['token_type_ids'].shape, labels.shape

(torch.Size([64, 60]),
 torch.Size([64, 60]),
 torch.Size([64, 60]),
 torch.Size([64]))

In [6]:
config = BertConfig.from_pretrained('kykim/bert-kor-base')
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "embedding_size": 768,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.51.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 42000
}

In [35]:
labels

tensor([237, 158, 229, 107,  68, 158,  41, 242,  34,  86, 203, 146, 180,  91,
         94, 146, 158, 191, 214,  61,  87,  61,  91,  91,  21,  49, 184, 173,
        125,  62,  22, 182,  20,  55, 151, 214,  47, 125,  17,  33,  33,  12,
         32, 103, 123,  52,  24, 207, 131, 212, 122,  39, 229,  35,  60, 225,
        214, 178, 214,  87, 107, 103,  67, 229])

In [7]:
model_bert = BertModel.from_pretrained('kykim/bert-kor-base').to(device)
model_bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(42000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [28]:
outputs = model_bert(**inputs)
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [29]:
outputs['last_hidden_state'].shape, outputs['pooler_output'].shape

(torch.Size([64, 60, 768]), torch.Size([64, 768]))

In [30]:
last_hidden_state = outputs['last_hidden_state']
print(last_hidden_state.shape)
print(last_hidden_state[:, 0, :])

torch.Size([64, 60, 768])
tensor([[-0.4606, -1.0344, -1.2719,  ..., -0.3460, -0.2235,  0.0035],
        [-0.5401, -0.3044,  0.4283,  ...,  0.0631,  0.2842,  1.4257],
        [ 0.2831, -0.6885, -0.1150,  ...,  0.0238, -0.3128,  0.8434],
        ...,
        [ 0.3198, -0.3580,  0.5679,  ..., -0.6770, -0.0985,  0.4845],
        [-0.2903, -0.6429,  0.2678,  ..., -0.1045,  0.1154,  0.7202],
        [-0.0439, -0.6012,  0.2019,  ...,  0.1405, -0.1838,  0.8078]],
       device='cuda:0', grad_fn=<SliceBackward0>)


In [31]:
pooler_output = outputs['pooler_output']
print(pooler_output.shape)
print(pooler_output)

torch.Size([64, 768])
tensor([[ 0.2271, -0.0198, -0.9998,  ...,  0.6436,  0.6461,  0.7775],
        [ 0.9827,  0.4934, -0.9986,  ...,  0.5567,  0.8611,  0.6830],
        [ 0.9869,  0.1407, -0.9919,  ...,  0.8425,  0.9108,  0.3534],
        ...,
        [ 0.9181,  0.0185, -0.9979,  ...,  0.0028,  0.7621,  0.4888],
        [ 0.9240,  0.4419, -0.9985,  ...,  0.4572,  0.8132,  0.6959],
        [ 0.9377,  0.3259, -0.9560,  ...,  0.1770,  0.8478,  0.6417]],
       device='cuda:0', grad_fn=<TanhBackward0>)


In [32]:
fc = nn.Linear(768, 247)
fc.to(device)
fc_output = fc(last_hidden_state[:, 0, :])
print(fc_output.shape)
print(fc_output.argmax(dim=1))

torch.Size([64, 247])
tensor([180, 180, 180, 180, 180,  76, 180, 180, 180, 180, 106, 180, 180,  76,
         23, 180, 180, 180, 180, 180, 180, 206, 180, 180, 180, 180, 180, 180,
        180,  32, 180, 180, 180, 180, 206, 180, 180, 180, 106, 180, 180, 180,
        180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180,
        150, 180, 180, 180,  32, 180, 180, 180], device='cuda:0')


In [33]:
class CustomBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        # 부모클래스 초기화
        super(CustomBertModel, self).__init__()
        # 사전학습 모델 지정
        self.bert = BertModel.from_pretrained(bert_pretrained)
        # dropout 설정
        self.dr = nn.Dropout(p=dropout_rate)
        # 최종 출력층 정의
        self.fc = nn.Linear(768, 247)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        # 입력을 pre-trained bert model 로 대입
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # 결과의 last_hidden_state 가져옴
        last_hidden_state = output['last_hidden_state']
        # last_hidden_state[:, 0, :]는 [CLS] 토큰을 가져옴
        x = self.dr(last_hidden_state[:, 0, :])
        # FC 을 거쳐 최종 출력
        x = self.fc(x)
        return x

In [6]:
class ImprovedBertModel(nn.Module):
    def __init__(self, bert_pretrained, num_classes=247, dropout_rate=0.3):
        super(ImprovedBertModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_pretrained)
        
        # 더 복잡한 분류기 추가
        self.classifier = nn.Sequential(
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs['last_hidden_state'][:, 0, :]
        logits = self.classifier(pooled_output)
        return logits

In [36]:
# bert = CustomBertModel('kykim/bert-kor-base')
bert = ImprovedBertModel('kykim/bert-kor-base')
bert.to(device)

ImprovedBertModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [37]:
# loss 정의: CrossEntropyLoss
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저 정의: bert.paramters()와 learning_rate 설정
optimizer = optim.Adam(bert.parameters(), lr=1e-5)

In [38]:
def model_train(model, data_loader, loss_fn, optimizer, device):
    # 모델을 훈련모드로 설정합니다. training mode 일 때 Gradient 가 업데이트 됩니다. 반드시 train()으로 모드 변경을 해야 합니다.
    model.train()
    
    # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
    running_loss = 0
    corr = 0
    counts = 0
    
    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합니다.
    prograss_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)
    
    # mini-batch 학습을 시작합니다.
    for idx, (inputs, labels) in enumerate(prograss_bar):
        # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
        inputs = {k:v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)
        
        # 누적 Gradient를 초기화 합니다.
        optimizer.zero_grad()
        
        # Forward Propagation을 진행하여 결과를 얻습니다.
        output = model(**inputs)
        
        # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
        loss = loss_fn(output, labels)
        
        # 오차역전파(Back Propagation)을 진행하여 미분 값을 계산합니다.
        loss.backward()
        
        # 계산된 Gradient를 업데이트 합니다.
        optimizer.step()
        
        # output의 max(dim=1)은 max probability와 max index를 반환합니다.
        # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
        _, pred = output.max(dim=1)
        
        # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
        # 합계는 corr 변수에 누적합니다.
        corr += pred.eq(labels).sum().item()
        counts += len(labels)
        
        # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
        # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
        # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
        running_loss += loss.item() * labels.size(0)
        
        # 프로그레스바에 학습 상황 업데이트
        prograss_bar.set_description(f"training loss: {running_loss/(idx+1):.5f}, training accuracy: {corr / counts:.5f}")
        
    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    acc = corr / len(data_loader.dataset)
    
    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader.dataset), acc

In [39]:
def model_evaluate(model, data_loader, loss_fn, device):
    # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다. 
    # dropout과 같은 layer의 역할 변경을 위하여 evaluation 진행시 꼭 필요한 절차 입니다.
    model.eval()
    
    # Gradient가 업데이트 되는 것을 방지 하기 위하여 반드시 필요합니다.
    with torch.no_grad():
        # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
        corr = 0
        running_loss = 0
        
        # 배치별 evaluation을 진행합니다.
        for inputs, labels in data_loader:
            # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
            inputs = {k:v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)
            
            # 모델에 Forward Propagation을 하여 결과를 도출합니다.
            output = model(**inputs)
            
            # output의 max(dim=1)은 max probability와 max index를 반환합니다.
            # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
            _, pred = output.max(dim=1)
            
            # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
            # 합계는 corr 변수에 누적합니다.
            corr += torch.sum(pred.eq(labels)).item()
            
            # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
            # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
            # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
            running_loss += loss_fn(output, labels).item() * labels.size(0)
        
        # validation 정확도를 계산합니다.
        # 누적한 정답숫자를 전체 데이터셋의 숫자로 나누어 최종 accuracy를 산출합니다.
        acc = corr / len(data_loader.dataset)
        
        # 결과를 반환합니다.
        # val_loss, val_acc
        return running_loss / len(data_loader.dataset), acc

In [ ]:
# checkpoint로 저장할 모델의 이름을 정의
model_name = 'bert-kor-mecab_all'
time_date = '2505071730'
save_path  = f'./saved_model/{model_name}{time_date}'
os.makedirs(save_path, exist_ok=True)

num_epochs = 10
min_loss = np.inf

history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

# Epoch 별 훈련 및 검증을 수행합니다.
for epoch in range(num_epochs):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    train_loss, train_acc = model_train(bert, train_loader, loss_fn, optimizer, device)

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = model_evaluate(bert, test_loader, loss_fn, device)

    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(bert.state_dict(), os.path.join(save_path, f'{model_name}{time_date}.pth'))
    
    # Epoch 별 결과를 출력합니다.
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

# 학습 기록 저장
pickle.dump(history, open(os.path.join(save_path, f'{model_name}{time_date}_history.pkl'), 'wb'))

  0%|          | 0/11366 [00:00<?, ?batch/s]

In [33]:
bert.load_state_dict(torch.load(f'{model_name}.pth'))

<All keys matched successfully>

In [ ]:
# 모델 가중치 저장
torch.save(bert.state_dict(), './saved_model/bert-kor-base2505042240.pth')

# 모델 구성 저장
with open('./saved_model/bert-kor-base2505042240/bert-kor-base2505042240_config.json', 'w') as f:
    f.write(config.to_json_string())

In [20]:
bert = ImprovedBertModel('kykim/bert-kor-base')
bert.load_state_dict(torch.load('./saved_model/bert-kor-mecab_all_2505071730/bert-kor-mecab_all_2505071730.pth'))
bert.to(device)
bert.eval() # 평가모드

ImprovedBertModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [21]:
tokenizer = BertTokenizerFast.from_pretrained('kykim/bert-kor-base')
labels = pickle.load(open('./processed_data/category_mapping.pkl', 'rb'))
labels = {v: k for k, v in labels.items()}
labels

{'0': 'PC방',
 '1': '가구',
 '2': '가발',
 '3': '가방',
 '4': '가스 충전소',
 '5': '가전제품',
 '6': '가전제품 수리',
 '7': '가정용 연료',
 '8': '가죽/가방/신발 수선',
 '9': '가축 사료',
 '10': '간이 음식점(기타)',
 '11': '개인/가정용품 수리(기타)',
 '12': '건강보조식품',
 '13': '건물 및 토목 엔지니어링 서비스',
 '14': '건설/건축자재',
 '15': '건설기계/장비 대여업',
 '16': '건어물/젓갈',
 '17': '건축 설계 및 관련 서비스',
 '18': '건축물 일반 청소',
 '19': '결혼 상담 서비스',
 '20': '경양식',
 '21': '경영 컨설팅',
 '22': '고용 알선업',
 '23': '곡물/곡분',
 '24': '골프 연습장',
 '25': '곱창 전골/구이',
 '26': '공인노무사',
 '27': '공인회계사',
 '28': '광고 대행',
 '29': '광고 매체 판매',
 '30': '광고물 설계/제작업',
 '31': '교육기관(기타)',
 '32': '교육컨설팅',
 '33': '구내식당',
 '34': '국/탕/찌개류',
 '35': '국수/칼국수',
 '36': '기념품점',
 '37': '기숙사/고시원',
 '38': '기타 개인/가정용품 대여업',
 '39': '기타 건설/건축자재',
 '40': '기타 광고 관련 서비스',
 '41': '기타 교육지원 서비스',
 '42': '기타 기술/직업 훈련학원',
 '43': '기타 동남아식 전문',
 '44': '기타 법무관련 서비스',
 '45': '기타 사무 지원 서비스',
 '46': '기타 산업용 기계/장비 대여업',
 '47': '기타 서양식 음식점',
 '48': '기타 스포츠시설 운영업',
 '49': '기타 엔지니어링 서비스',
 '50': '기타 여행 보조/예약 서비스',
 '51': '기타 예술/스포츠 교육기관',
 '52': 

In [22]:
class CustomPredictor():
    def __init__(self, model, tokenizer, labels: dict):
        self.model = model
        self.tokenizer = tokenizer
        self.labels = labels
        
    def predict(self, sentence):
        # 토큰화 처리
        tokens = self.tokenizer(
            str(sentence),                # 1개 문장 
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            max_length=60,          # 최대 길이 60
            add_special_tokens=True  # 스페셜 토큰 적용
        )
        tokens.to(device)
        prediction = self.model(**tokens)
        prediction = F.softmax(prediction, dim=1)
        output = prediction.argmax(dim=1).item()
        prob = prediction.max(dim=1)[0].item() * 100
        result = self.labels[str(output)]
        return result, prob

In [23]:
predictor = CustomPredictor(bert, tokenizer, labels)

for i in range(10):
    query = test_df.iloc[i]['store']
    pred, acc = predictor.predict(query)
    print(f'{query} -> {pred} ({acc:.2f}%)')
query = "스타벅스R리저브강남속편한치과건너편점"
pred = predictor.predict(query)
print(query, pred)

캐치캐치아이스 -> 광고 대행 (16.63%)
아엠 -> 여성 의류 (10.98%)
새틴 -> 경영 컨설팅 (6.80%)
네일즈247 -> 네일숍 (98.66%)
커피랑도서관대구본동점 -> 카페 (69.94%)
꿈꾸는요리사 -> 기타 기술/직업 훈련학원 (30.25%)
메타교육 -> 입시·교과학원 (57.72%)
나선재 -> 경영 컨설팅 (29.49%)
평화이앤씨주 -> 건물 및 토목 엔지니어링 서비스 (12.41%)
약령 -> 건강보조식품 (53.39%)
스타벅스R리저브강남속편한치과건너편점 ('카페', 94.3593680858612)


In [26]:
def batch_predict(model, dataloader, labels_dict, device):
    """배치 단위로 예측 수행"""
    model.eval()
    results = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader):
            # 디바이스로 데이터 이동
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)
            
            # 예측 수행
            outputs = model(**inputs)
            probs = F.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)
            confidence = torch.max(probs, dim=1)[0]
            
            # 결과 저장
            for i in range(len(labels)):
                pred_class = labels_dict[str(preds[i].item())]
                true_class = labels_dict[str(labels[i].item())]
                results.append({
                    'query': test_df.iloc[i]['store'],
                    'pred': pred_class,
                    'label': true_class,
                    'prob': f'{confidence[i].item()*100:.2f}%'
                })
    
    return pd.DataFrame(results)

# 평가용 데이터로더 설정 (배치 크기 증가)
test_loader = DataLoader(test_data, batch_size=256, shuffle=False, num_workers=0)

# 배치 예측 실행
test_query_df = batch_predict(bert, test_loader, labels, device)

100%|██████████| 1421/1421 [16:36<00:00,  1.43it/s]


In [27]:
test_query_df

,query,pred,label,prob
0,캐치캐치아이스,슈퍼마켓,슈퍼마켓,37.68%
1,아엠,여성 의류,명함/간판/광고물 제작,16.42%
2,새틴,백반/한정식,여관/모텔,9.23%
3,네일즈247,네일숍,네일숍,97.87%
4,커피랑도서관대구본동점,독서실/스터디 카페,독서실/스터디 카페,99.90%
...,...,...,...,...
363693,영구당구클럽,자동차 대여업,의류 대여업,56.40%
363694,대세칼국수,요양병원,요양병원,96.43%
363695,지에스25제주용흥점,떡/한과,떡/한과,99.71%
363696,한터합동공인중개사사무소,가방,여성 의류,15.11%


In [28]:
score = test_query_df['pred'].eq(test_query_df['label']).sum() / len(test_query_df) * 100
score

np.float64(52.467156816919534)

In [29]:
# Error Analysis
error_df = test_query_df[test_query_df['pred'] != test_query_df['label']].copy()
error_df

,query,pred,label,prob
1,아엠,여성 의류,명함/간판/광고물 제작,16.42%
2,새틴,백반/한정식,여관/모텔,9.23%
5,꿈꾸는요리사,펜션,빵/도넛,7.46%
6,메타교육,입시·교과학원,요가/필라테스 학원,59.08%
7,나선재,백반/한정식,펜션,6.79%
...,...,...,...,...
363689,꽃남자미용실,백반/한정식,컴퓨터/노트북/프린터 수리,9.23%
363691,어린왕자QStation,반찬/식료품,채소/과일,53.86%
363692,션텍스,카페,임시/일용 인력 공급업,14.74%
363693,영구당구클럽,자동차 대여업,의류 대여업,56.40%


In [30]:
pickle.dump(test_query_df, open('./saved_model/bert-kor-mecab_all_2505071730/test_query_df.pkl', 'wb'))
pickle.dump(error_df, open('./saved_model/bert-kor-mecab_all_2505071730/error_df.pkl', 'wb'))

In [31]:
test_query_df = pickle.load(open('./saved_model/bert-kor-mecab_all_2505071730/test_query_df.pkl', 'rb'))
error_df = pickle.load(open('./saved_model/bert-kor-mecab_all_2505071730/error_df.pkl', 'rb'))
error_df.head(20)

,query,pred,label,prob
1,아엠,여성 의류,명함/간판/광고물 제작,16.42%
2,새틴,백반/한정식,여관/모텔,9.23%
5,꿈꾸는요리사,펜션,빵/도넛,7.46%
6,메타교육,입시·교과학원,요가/필라테스 학원,59.08%
7,나선재,백반/한정식,펜션,6.79%
8,평화이앤씨주,건물 및 토목 엔지니어링 서비스,건축 설계 및 관련 서비스,44.93%
9,약령,백반/한정식,경양식,9.23%
11,웅진NS,슈퍼마켓,서점,3.88%
12,삼춘종합집수리,철물/공구,인테리어 디자인업,54.48%
13,예술창작소숲,전시/컨벤션/행사 대행 서비스,예술품,19.83%


In [32]:
print(classification_report(test_query_df['label'], test_query_df['pred'], digits=4, zero_division=0))

                      precision    recall  f1-score   support

                 PC방     0.5088    0.5014    0.5051      1037
                  가구     0.6776    0.5046    0.5785      1508
                  가발     0.4000    0.0552    0.0970       145
                  가방     0.5855    0.3145    0.4092       566
              가스 충전소     0.7672    0.5443    0.6369       327
                가전제품     0.3470    0.2545    0.2937      1163
             가전제품 수리     0.4204    0.4386    0.4293      1662
              가정용 연료     0.7988    0.7167    0.7556       759
         가죽/가방/신발 수선     0.6538    0.2537    0.3656       134
               가축 사료     0.4828    0.0547    0.0982       256
          간이 음식점(기타)     0.4907    0.0911    0.1536      1153
      개인/가정용품 수리(기타)     0.5447    0.3272    0.4088      1247
              건강보조식품     0.5657    0.5164    0.5399      2351
   건물 및 토목 엔지니어링 서비스     0.2933    0.3448    0.3170      1160
             건설/건축자재     0.5845    0.4213    0.4897       197
       